## quant-econ Solutions: Discrete Dynamic Programming

Solutions for http://quant-econ.net/jl/discrete_dp.html

Written jointly with [Diasuke Oyama](https://github.com/oyamad) and [Max Huber](https://github.com/MaximilianJHuber)

The exercise is to replicate numerically the analytical solution for the benchmark model in  [this lecture](http://quant-econ.net/jl/optgrowth.html) of quant-econ, using `DiscreteDP`.

## Setup

First some basic imports:

In [1]:
using QuantEcon
using Plots

We're going to use the `PlotlyJS.jl` backend for plots.  If you don't have this installed you can install it [try other options](https://juliaplots.github.io/backends/).

In [2]:
plotlyjs()

Plots.PlotlyJSBackend()

Details of the model can be found in [the lecture](http://quant-econ.net/jl/optgrowth.html).  As in the lecture,
we let $f(k) = k^{\alpha}$ with $\alpha = 0.65$, $u(c) = \log c$, and $\beta = 0.95$.

In [3]:
alpha = 0.65
f(k) = k.^alpha
u(x) = log(x)
beta = 0.95

0.95

Here we want to solve a finite state version of the continuous state model above.
We discretize the state space into a grid of size `grid_size=500`,
from $10^{-6}$ to `grid_max=2`.

In [4]:
grid_max = 2
grid_size = 500
grid = linspace(1e-6, grid_max, grid_size)

linspace(1.0e-6,2.0,500)

We choose the action to be the amount of capital to save for the next period
(the state is the capical stock at the beginning of the period).
Thus the state indices and the action indices are both `0`, ..., `grid_size-1`.
Action (indexed by) `a` is feasible at state (indexed by) `s` if and only if
`grid[a] < f([grid[s])`
(zero consumption is not allowed because of the log utility).

Thus the Bellman equation is:
$$
v(k) = \max_{0 < k' < f(k)} u(f(k) - k') + \beta v(k'),
$$
where $k'$ is the capital stock in the next period.

The transition probability array `Q` will be highly sparse
(in fact it is degenerate as the model is deterministic),
so we formulate the problem with state-action pairs, to represent `Q` in
sparse matrix format.

We first construct indices for state-action pairs:

In [5]:
C = reshape(f(grid),grid_size,1).-reshape(grid,1,grid_size)
coord = repmat(collect(1:grid_size),1,grid_size) #coordinate matrix 
s_indices = coord[C.>0]
a_indices = transpose(coord)[C.>0]
L = length(a_indices)

118841

In [6]:
s_indices

118841-element Array{Int64,1}:
   1
   2
   3
   4
   5
   6
   7
   8
   9
  10
  11
  12
  13
   ⋮
 498
 499
 500
 496
 497
 498
 499
 500
 498
 499
 500
 500

Now let's set up $R$ and $Q$

In [7]:
R = u(C[C.>0]);

In [8]:
Q = spzeros(L,grid_size)

for i in 1:L
  Q[i,a_indices[i]] = 1
end

We're now in a position to create an instance of `DiscreteDP` corresponding to the growth model.

In [9]:
using QuantEcon
ddp = DiscreteDP(R, Q, beta, s_indices, a_indices);

### Solving the Model

Let's try solving via policy function iteration.

In [10]:
results = solve(ddp, PFI);

In [11]:
v, sigma, num_iter = results.v, results.sigma, results.num_iter

num_iter

10

Let us compare the solution of the discrete model with the exact solution of the original continuous model.  Here's the exact solution:

In [12]:
c = f(grid) - grid[sigma]

ab = alpha * beta
c1 = (log(1 - alpha * beta) + log(alpha * beta) * alpha * beta / (1 - alpha * beta)) / (1 - beta)
c2 = alpha / (1 - alpha * beta)

v_star(k) = c1 + c2 * log(k)
c_star(k) = (1 - alpha * beta) * k.^alpha

c_star (generic function with 1 method)

Let's plot the value functions.

In [13]:
plot(grid, [v v_star(grid)], ylim=(-40, -32), lw=2, label=["discrete" "continuous"])

[Plots.jl] Initializing backend: plotlyjs


They are barely distinguishable (although you can see the difference if you zoom).

Now let's look at the discrete and exact policy functions for consumption.

In [14]:
plot(grid, [c c_star(grid)], lw=2, label=["discrete" "continuous"])

These functions are again close, although some difference is visible and becomes more obvious as you zoom.  Here are some statistics:

In [15]:
maximum(abs(v - v_star(grid)))

121.49819147053377

This is a big error, but most of the error occurs at the lowest gridpoint.  Otherwise the fit is reasonable:

In [16]:
maximum(abs(v - v_star(grid))[2:end])

0.012681735127422655

The policy function fit is good:

In [17]:
maximum(abs(c - c_star(grid)))

0.003826523100009971

The value function is monotone, as expected:

In [18]:
all(diff(v).>=0)

true

Comparison of the solution methods
--

Let's try different solution methods.  The results below show that policy function iteration and modified policy function iteration are much faster that value function iteration.

In [19]:
@time results = solve(ddp, PFI);

  0.707917 seconds (631 allocations: 108.354 MB, 2.10% gc time)


In [20]:
@time res1 = solve(ddp, VFI, max_iter=500, epsilon=1e-4);

  7.805179 seconds (209.16 k allocations: 813.698 MB, 0.70% gc time)


In [21]:
res1.num_iter

294

In [22]:
sigma == res1.sigma

true

In [23]:
@time res2 = solve(ddp, MPFI, max_iter=500, epsilon=1e-4);

  0.991594 seconds (242.82 k allocations: 662.363 MB, 5.15% gc time)


In [24]:
res2.num_iter

16

In [25]:
sigma == res2.sigma

true

Replication of the figures
--

Let's visualize convergence of value function iteration, as in the lecture.

In [26]:
w_init = 5 * log(grid) - 25  # Initial condition
n = 50

ws = []
colors = []
w = w_init
for i in 0:n-1
    w = bellman_operator(ddp, w)
    push!(ws, w)
    push!(colors, RGBA(0, 0, 0, i/n))
end

plot(grid, 
    w_init, 
    ylims=(-40, -20), 
    lw=2,
    xlims=(minimum(grid), maximum(grid)), 
    label="initial condition")

plot!(grid, ws,  label="", color=colors', lw=2)
plot!(grid, v_star(grid), label="true value function", color=:red, lw=2)


We next plot the consumption policies along the value iteration.   First we write a function to generate the and record the policies at given stages of iteration.

In [27]:

function compute_policies(n_vals...)
    c_policies = []
    w = w_init
    for n in 1:maximum(n_vals)
        w = bellman_operator(ddp, w)
        if n in n_vals
            sigma = compute_greedy(ddp, w)
            c_policy = f(grid) - grid[sigma] 
            push!(c_policies, c_policy)
        end
    end

    return c_policies
end

compute_policies (generic function with 1 method)

Now let's generate the plots.

In [28]:

true_c = c_star(grid)
c_policies = compute_policies(2, 4, 6)
plot_vecs = [c_policies[1] c_policies[2] c_policies[3] true_c true_c true_c]
l1 = "approximate optimal policy"
l2 = "optimal consumption policy"
labels = [l1 l1 l1 l2 l2 l2]
plot(grid, 
    plot_vecs, 
    xlim=(0, 2), 
    ylim=(0, 1), 
    layout=(3, 1), 
    lw=2,
    label=labels,
    size=(600, 800),
    title=["2 iterations" "4 iterations" "6 iterations"])

Dynamics of the capital stock
--

Finally, let us work on [Exercise 2](http://quant-econ.net/py/optgrowth.html#exercise-2),
where we plot the trajectories of the capital stock for three different discount factors,
$0.9$, $0.94$, and $0.98$, with initial condition $k_0 = 0.1$.

In [29]:
discount_factors = (0.9, 0.94, 0.98)
k_init = 0.1

k_init_ind = findfirst(collect(grid) .>= k_init, true)

sample_size = 25

ddp0 = DiscreteDP(R, Q, beta, s_indices, a_indices)
k_paths = []
labels = []

for beta in discount_factors
    ddp0.beta = beta
    res0 = solve(ddp0,PFI)
    k_path_ind = simulate(res0.mc,sample_size,k_init_ind)
    k_path = grid[k_path_ind.+1]
    push!(k_paths, k_path)
    push!(labels, "beta = $beta")
end

plot(k_paths, 
    xlabel="time", 
    ylabel="capital", 
    ylim=(0.1, 0.3), 
    lw=2,
    markershape=:circle,
    label=labels')